In [1]:
import os
from os import listdir
from PIL import Image
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from keras.models import load_model
import numpy as np

import pickle
import cv2
from openpyxl import Workbook
import datetime
import keras_facenet

In [2]:
book = Workbook()
sheet = book.active

In [3]:
from mtcnn.mtcnn import MTCNN

In [4]:
detector = MTCNN()
model_path ="facenet_keras.h5"
#HaarCascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))
MyFaceNet = load_model(model_path)
facenet = keras_facenet.FaceNet('20180402-114759')

In [5]:
folder='C:/Users/nelso/FaceRecognition_main/My_project/Train_Images/' #create each folder for per classes
database = {}                                                         # to save the annots
for foldername in listdir(folder):
    folderdir = 'C:/Users/nelso/FaceRecognition_main/My_project/Train_Images/'+foldername+'/'
    
    for filename in listdir(folderdir):

        path = folderdir + filename
        gbr1 = cv2.imread(folderdir + filename)

        faces = detector.detect_faces(gbr1)

        for result in faces:

            if len(faces)>0:
                x1, y1, width, height = result["box"]
            else:
                x1, y1, width, height = 1, 1, 10, 10

            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height

            gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
            gbr = Image.fromarray(gbr)                  
            gbr_array = asarray(gbr)

            face = gbr_array[y1:y2, x1:x2]                        

            face = Image.fromarray(face)                       
            face = face.resize((160,160))
            face = asarray(face)

            face = face.astype('float32')
            mean, std = face.mean(), face.std()
            face = (face - mean) / std

            face = expand_dims(face, axis=0)
            signature = MyFaceNet.predict(face)


            database[os.path.splitext(filename)[0]]=signature
    # MyFaceNet.save("training_data.yml")
    # database.keys()

1/1 [==============================] - 0s 18ms/step


1/1 [==============================] - 0s 61ms/step


In [6]:
def findFaces(data):
    gbr1 = data                                      #(use this for video capturing)js_to_image(data)
    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)                  # konversi dari OpenCV ke PIL
    gbr_array = asarray(gbr)

    wajah = detector.detect_faces(gbr1)
    final_set = set()
#     extracted_faces = 0
    
    for result in wajah:
        x1, y1, w, h = result['box']
    #for (x1,y1,w,h) in wajah:
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + w, y1 + h

        face = gbr_array[y1:y2, x1:x2]
#         #for extracting images from output
        
#         cv2.imshow("detected.jpg",face)
        
#         cv2.imwrite("c:/Users/nelso/FaceRecognition_main/My_project/main/Extracted/"+str(extracted_faces)+".jpg", gbr1)
#         extracted_faces+=1


        face = Image.fromarray(face)                       
        face = face.resize((160,160))
        face = asarray(face)

        face = face.astype('float32')
        mean, std = face.mean(), face.std()
        face = (face - mean) / std

        face = expand_dims(face, axis=0)
        signature = MyFaceNet.predict(face)

        min_dist=100
        identity=' '
        for key, value in database.items() :
            
            dist = np.linalg.norm(value-signature)

            if dist < min_dist:
                min_dist = dist
                identity = key
            text=""
            numbers=""
            res=[]
            for i in identity:
                if(i.isdigit()):
                    numbers+=i
                else:
                    text+=i
            res.append(text)
            res.append(numbers)
        final_set.add(text)
        cv2.putText(gbr1,identity, (x1,y1),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1, cv2.LINE_AA)
        cv2.rectangle(gbr1,(x1,y1),(x2,y2), (0,255,0), 2)
        print(identity)
       
#print(final_set)


      #EXTRACTING IMAGES
    

    cv2.imshow("image",gbr1)
    print(final_set)
    # keep the window open until we press a key
    cv2.waitKey(0)
    # close the window
    cv2.destroyAllWindows()
    return final_set

In [7]:
img = cv2.imread("C:/Users/nelso/FaceRecognition_main/My_project/Test_Images/1 (25).jpg") #file
  # cv2_imshow(img)
output = findFaces(img)

#extracting total students from main folder

total_stud_folder='C:/Users/nelso/FaceRecognition_main/My_project/Train_Images/'
students_name_exl = set()
for folder_name in listdir(total_stud_folder):
    students_name_exl.add(folder_name)




attendance_exl = []
for names in students_name_exl:
    attendance_exl.append(names)
attendance_exl

now= datetime.datetime.now()
today=now.day
month=now.month
year = now.year

#index in excel
sheet["A1"] = "Students_name"
for row in range(1,30):
    if row <= len(attendance_exl):
        sheet.cell(row=row+1,column=1).value=attendance_exl[row-1]

#casting outputs to Type List
list_ouput = []
for i in output:
    list_ouput.append(i)
list_ouput

#markings in excel
markings = {}
for i in range(len(attendance_exl)):
    if attendance_exl[i] not in list_ouput:
        markings[i]="absent"
    else:
        markings[i]="present"

book.save("stud.xlsx")
  




#marking values in second column with date

sheet["B1"] = "{}/{}/{}".format(today,month,year)
markings_list = []
for i in markings:
    markings_list.append(markings[i])
    
sheet["A1"] = "Students_name"
for row in range(1,30):
    if row <= len(markings_list):
        sheet.cell(row=row+1,column=2).value=markings_list[row-1]
book.save("stud.xlsx")



1/1 [==============================] - 0s 57ms/step
Harry
1/1 [==============================] - 0s 55ms/step
Sujan
1/1 [==============================] - 0s 54ms/step
Badri
1/1 [==============================] - 0s 52ms/step
Kavi1
{'Sujan', 'Badri', 'Harry', 'Kavi'}


In [ ]:
#dumping the model in current dir
myfile = open("data.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()